In [1]:
import ee

ee.Authenticate()

ee.Initialize()

Enter verification code:  4/1AfJohXkTgBd0kN2HNU1MYYrp6auCGT6r_K8utWK7WMCezWS3pMs7-mw5MqM



Successfully saved authorization token.


In [2]:
# Define the corners of the large ROI
top_left = (70, 45)
bottom_right = (110, 23)

# Define the grid size
cols = 5  # Number of columns
rows = 4  # Number of rows

# Calculate the step size for each grid cell
width_step = (bottom_right[0] - top_left[0]) / cols
height_step = (top_left[1] - bottom_right[1]) / rows

In [3]:
def create_grid(top_left, bottom_right, width_step, height_step, cols, rows):
    grid = []
    for i in range(cols):
        for j in range(rows):
            # Calculate corners of the cell
            left = top_left[0] + i * width_step
            right = left + width_step
            top = top_left[1] - j * height_step
            bottom = top - height_step

            # Create a rectangle for this cell
            cell = ee.Geometry.Rectangle([left, bottom, right, top])
            grid.append(cell)
    return grid

grid = create_grid(top_left, bottom_right, width_step, height_step, cols, rows)


In [4]:
era5_land_temp = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").select('temperature_2m')
era5_land_precip = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").select('total_precipitation')

filtered_temp = era5_land_temp.filterDate('2012-01-01', '2013-12-31')
filtered_precip = era5_land_precip.filterDate('2012-01-01', '2013-12-31')


In [ ]:
for i, cell in enumerate(grid):
    for year in range(2012, 2014):
        # Temperature
        temp_image = filtered_temp.filter(ee.Filter.calendarRange(year, year, 'year')).mean().clip(cell)
        temp_task = ee.batch.Export.image.toDrive(**{
            'image': temp_image,
            'description': f'Temperature_{year}_Section_{i}',
            'folder': 'EarthEngineData',
            'scale': 9000,  # ERA5-Land native resolution
            'region': cell,
            'fileFormat': 'GeoTIFF'
        })
        temp_task.start()

        # Precipitation
        precip_image = filtered_precip.filter(ee.Filter.calendarRange(year, year, 'year')).sum().clip(cell)
        precip_task = ee.batch.Export.image.toDrive(**{
            'image': precip_image,
            'description': f'Precipitation_{year}_Section_{i}',
            'folder': 'EarthEngineData',
            'scale': 9000,  # ERA5-Land native resolution
            'region': cell,
            'fileFormat': 'GeoTIFF'
        })
        precip_task.start()
